# ニューラルネットワークの学習
## 損失関数
- 損失関数は教師データに対してどれだけ適合してないかを表す　
- パラメータを変更しても認識精度は変化しづらく不連続な性質があるが、損失関数は連続的であるので指標にできる
- 損失関数の例
    - 2乗和誤差
    - 交差エントロピー誤差
        - log e
        - 確率分布がどれくらいはね

## 勾配法
- 傾きを求めて一定の距離を移動して、その点でも傾きを求めるのを繰り返す
- 注意として微分が０だとしても最小値とは限らない
- 一定の距離を学習率と呼ばれている


In [10]:
import sys
sys.path.append('./deep-learning-from-scratch')

import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.rand(2, 3) # ガウス分布
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

net = simpleNet()
print(f"重み: {net.W}")

x = np.array([0.6, 0.9])
p = net.predict(x)
print(f"予測: {p}")

np.argmax(p)

t = np.array([0, 0, 1]) # 正解ベクトル
print(f"損失関数: {net.loss(x, t)}")

dW = numerical_gradient(lambda w: net.loss(x, t), net.W)
print(dW)


重み: [[0.81712406 0.99336792 0.80548288]
 [0.74296342 0.23464375 0.46333048]]
予測: [1.15894151 0.80720013 0.90028716]
損失関数: 1.1651173773773003
[[ 0.24237012  0.17049818 -0.4128683 ]
 [ 0.36355518  0.25574726 -0.61930244]]


## 学習アルゴリズムの実装
以下のステップで学習していく
1. ミニバッチ
    - 膨大の訓練データから一部のデータを取り出してパラメータを決定
1. 勾配の算出
1. パラメータの更新
1. 上記ステップを繰り返す

※バッチを無作為に選び勾配降下法を使うことから、**確率的勾配降下法**(stochastic gradient descent: SGD)と呼ぶ

In [5]:
# ２層ニューラルネットワーク
import sys
sys.path.append('./deep-learning-from-scratch')

import numpy as np
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {
            'W1': weight_init_std * np.random.randn(input_size, hidden_size),
            'b1': np.zeros(hidden_size),
            'W2': weight_init_std * np.random.randn(hidden_size, output_size),
            'b2': np.zeros(output_size),
        }
    
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {
            "W1": numerical_gradient(loss_W, self.params["W1"]),
            "b1": numerical_gradient(loss_W, self.params["b1"]),
            "W2": numerical_gradient(loss_W, self.params["W2"]),
            "b2": numerical_gradient(loss_W, self.params["b2"]),
        }

        return grads


In [7]:
# ミニバッチの実装
import numpy as np
from dataset.mnist import load_mnist

(x_train , t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# ハイパーパラメータ
iters_num = 10_000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.numerical_gradient(x_batch, t_batch)

    for key in network.params.keys():
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
